#logging

In [ ]:
import logging
import logging.config
import json

LOG_CONFIG = {
    "version" : 1,
    "formatters": {
        "basic" :{
            "format" : "%(asctime)s:%(levelname)s:%(name)s:[%(filename)s] - %(message)s",
            "datefmt" : "%Y-%m-%d %H:%M:%S"
        },
        "detail": {
            "format" : "%(asctime)s:%(levelname)s:%(name)s:[%(filename)s Line %(lineno)s] - %(message)s",
            "datefmt" : "%Y-%m-%d %H:%M:%S"
        }
    },
    "handlers": {
        "console_info":{
            "class":"logging.StreamHandler",
            "level":"INFO",
            "formatter":"detail",
            "stream":"ext://sys/stdout"
        },
        "file_info":{
            "class":"logging.FileHandler",
            "level":"INFO",
            "formatter":"basic",
            "filename":"./logs/info.log"
        },
        "file_debug": {
            "class":"logging.handlers.RotatingFileHandler",
            "level" : "DEBUG",
            "formatter":"detail",
            "filename":"./logs/debug.log",
            "maxBytes" : 100*1000000, #100MB 이후 rotate
            "backupCount":5
        },
        "smtp_info":{
            "class":"logging.handlers.SMTPHandler",
            "level" : "INFO",
            "formatter":"basic",
            "mailhost":"smtp1.lguplus.co.kr",
            "fromaddr":"abas@lguplus.co.kr",
            "toaddrs":"kimsangho@lguplus.co.kr",
            "subject":"Logging Test"

        },
    },
    "loggers":{
        "abas_logger":{
            "level":"DEBUG",
            "handlers":["file_info","file_dubug"],
            "propagate":False
        },
        "lib":{
            "level":"DEBUG",
            "handlers":["file_info","file_dubug"],
            "propagate":False
        }
    }

logging.config.dictConfig(LOG_CONFIG)

logger = logging.getLogger('abas_logger')

#mail

In [ ]:
import lib.logger as my_log
import lib.mailer as mail

info_msg = "info logging message"
error_msg = "ERROR"

print(info_msg)
my_log.logger.info(info_msg)

print(error_msg_msg)
my_log.logger.error(error_msg)

my_log.logger.debug("Debug logging message")
mail.log_send_mail(filename=__file__,log_files_path=['./logs/info.log','./logs/debug.log'])
#logs 폴더 생성해야 함

#mailer

In [ ]:
import os
import smtplib

from email import encoders
from email.utils import formataddr
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import logging
logger = logging.getLogger(__name__)

class Mailer:
    def __init__(self,smtp_host:str, smtp_port:int,from_addr:str,to_addrs:list,debug=False,**kwargs):
        self.encoding = kwargs.get("encoding","UTF-8")

        self.host = smtp_host
        self.port = smtp_port
        self.from_addr = from_addr
        self.to_addr = to_addrs

        self.is_login = False
        self.exist_attachments = False

        #인증 계정
        if kwargs.get("username") is not None:
            self.set_auth(kwargs.get("username"),kwargs.get("password"))

        self.debug = debug is not False #디버그 모드

        #첨부파일로 전송 불가능한 확장자

        self.blocked_extensions=(
            ".ade",".adp",".apk",".appx",".appxbundle",".bat", ".cab",".chm",".cmd",".com",
            ".cpl",".dll",".dmg",".exe",".hta",".ins",".isp",".iso",".jar",".js",".jse",".lib",
            ".lnk",".mde",".msc",".msi",".msix",".msixbundle",".msp",".mst",".nsh",".pif",".ps1",
            ".scr",".sct",".shb",".sys",".vb",".vbe",".vbs",".vxd",".wsc",".wsf",".wsh"

        )

        def set_auth(self, username,password):
            self.is_login = True
            self.username = username
            self.password = password

        def write_mail(self, subject, body, body_type = "plain",attachments = None):
            '''
            content-type:
            -multipart/alternative : 평문 텍스트와 html과 같이 다른 포맷을 함께 보낸 메세지
            - multipart/mixed : 첨부파일이 포함된 메세지

            '''
            #첨부파일 여부
            self.exist_attachments = attachments is not None
            self.content_sub_type = "mixed" if self.exist_attachments else "alternative"

            #메일 콘텐츠 설정
            self.message = MIMEMultipart(self.content_sub_type)

            #받는 사람 주소 설정
            self.from_addr_fmt = formataddr(self.from_addr)
            self.to_addrs_fmt = ",".join([formataddr(addr) for addr in self.to_addrs])

            #메일 헤더 설정
            self.message.set_charset(self.encoding)
            self.message['From'] = self.from_addr_fmt
            self.message["To"] = self.to_addrs_fmt
            self.message["Subject"] = subject

            #메일 콘텐츠 내용
            self.message.attach(MIMEText(body,body_type, self.encoding))

            #메일 콘텐츠 첨부파일
            if self.exist_attachments:
                self.attach_files(attachments)

            return self

        def attach_files(self, attachments):
            '''
            content-disposition : 파일명 지정
            MIME type :  application/octect-stream : Binary Data
            REF :

            '''
            for attachment in attachments:
                attach_binary = MIMEBase("application","octect-stream")
                try:
                    binary = open(attachment,"rb").read()

                    attach_binary.set_payload(binary)
                    encoders.encode_base64(attach_binary)

                    filename = os.path.basename(attachment)
                    attach_binary.add_header("Content-Disposition","attachment", filename = (self.encoding, "",filname))
                    self.message.attach(attach_binary)

                except Exception as e:
                    logger.error(e)

        def send(self):
            session = None
            try:
                #메일 세션 생성
                session = smtplib.SMTP(self.host, self.port)
                session.set_debuglevel(self.debug)
                session.ehlo()

                #smtp 서버 계정 인증
                if self.is_login:
                    try:
                        session.starttls()
                        session.login(self.username, self.password)
                    except Exception as e:
                        logger.error(e)

                #메일 발송
                session.sendmail(self.from_addr_fmt, self.to_addrs_fmt, self.message.as_string())
                logger.info('Successfully sent the mail')

            except Exception as e:
                logger.error(e)

            finally:
                if session is not None:
                    session.quit()

def log_send_mail(filename:str, log_files_path:list):
    #smtp 서버 정보
    smtp_host = "smtp1.lguplus.co.kr"
    smtp_port = 25

    #메일 송수신자 정보
    from_addr = ('Sangho,kim'm"kimsangho@lguplus.co.kr")
    to_addrs = [('Sangho,kim'm"kimsangho@lguplus.co.kr")]

    #메일 제목
    subject = f"{filename}'s log"
    #메일 내용
    body = "mailer class send logs to you. check attachments"

    #첨부파일
    attachments = log_files_path
    mailer = Mailer(smtp_host, smtp_port, from_addr, to_addrs, debug=False)

    logger.info("write mail")
    mailer.write_mail(subject, body, 'html', attachments=attachments)
    logger.info("send mail")
    try:
        mailer.send()
    except Exception as e:
        logger.error("ERROR:",e)
